In [51]:
import numpy as np
from datetime import datetime
from kadlu.geospatial.data_sources import chs
from kadlu.geospatial.data_sources import era5
from kadlu.geospatial.data_sources import hycom
from kadlu.geospatial.data_sources import wwiii
from kadlu.geospatial.data_sources.fetch_util import database_cfg
from kadlu.geospatial.data_sources.fetch_util import dt_2_epoch

#from kadlu.geospatial.data_sources import fetch_util

conn, db = database_cfg()

In [ ]:
from importlib import reload
reload(wwiii)
reload(chs)

## Parameter Selection 

In [52]:
temp_salinity_source = hycom.Hycom()
bathysource = chs.Chs()
wavesource1 = era5.Era5()
wavesource2 = wwiii.Wwiii()

In [53]:
# parameter selection
start = datetime(2015, 3, 9, 13, 25, 0)
end   = datetime(2015, 3, 9, 19, 0, 0)
top, bottom = 0, 0 

# gulf st lawrence
south, west = 44.013, -70.429
north, east = 51.823, -55.977

# no wave data for gulf st lawrence :( 

# hudsons bay
#south, west = 51.388, -95.321
#north, east = 64.068, -75.703
# hudsons bay subset
#north, east = 55.039, -78.740

## Fetching

In [ ]:
temp_salinity_source.fetch_salinity(south=south, north=north, west=west, east=east, 
                      start=start, end=end, top=top, bottom=bottom)

temp_salinity_source.fetch_temp(south=south, north=north, west=west, east=east, 
                      start=start, end=end, top=top, bottom=bottom)

In [ ]:
bathysource.fetch_bathymetry(south=south, north=north, west=west, east=east)

In [ ]:
wavesource1.fetch_windwaveswellheight(south=south, north=north, 
                                      west=west, east=east, 
                                      start=start, end=end)

In [54]:
wavesource2.fetch_windwaveheight(south=south, north=north, 
                                 west=west, east=east, 
                                 start=start, end=end)

downloading multi_1.at_4m.hs.201503.grb2 from NOAA WaveWatch III...
preparing multi_1.at_4m.hs.201503.grb2 for the database...
formatting and inserting... from 2015-04-01 00:00:00...
processed and inserted 0 rows. 64723764 null values removed, 5460870 duplicate rows ignored


## Loading (example)

In [ ]:
temp, tlat, tlon, ttime, tdep = \
temp_salinity_source.load_temp(south=south, north=north, west=west, east=east, 
                               start=start, end=end, top=top, bottom=bottom)

salinity, slat, slon, stime, sdep = \
temp_salinity_source.load_temp(south=south, north=north, west=west, east=east, 
                               start=start, end=end, top=top, bottom=bottom)

In [ ]:
bathy, blat, blon = bathysource.load_bathymetry(south=south, north=north, west=west, east=east)

In [ ]:
wave, wlat, wlon, wtime = \
wavesource1.load_windwaveswellheight(south=south, north=north, 
                                     west=west, east=east, 
                                     start=start, end=end)

In [ ]:
wave, wlat, wlon, wtime = \
wavesource2.load_windwaveheight(south=south, north=north, 
                                west=west, east=east, 
                                start=start, end=end)

## SoundSpeed -- loading data

In [ ]:
from kadlu.geospatial.ocean import Ocean
from kadlu.sound.sound_speed import SoundSpeed 
from kadlu.geospatial.data_sources.fetch_util import storage_cfg

In [ ]:
step = 1 # 1 deg -> approx. 100km at equator 

# step through each 1x1 degree box in selected area 
for ymin in np.arange(south, north, step):
    for xmin in np.arange(west, east, step):
        xmax = xmin + step
        ymax = ymin + step
        
        # load bathymetry
        try:
            bathy, blat, blon = bathysource.load_bathymetry(south=ymin, north=ymax, west=xmin, east=xmax)
        except:
            print(f"no bathy data for {(ymin, xmin)}, skipping")
            continue
        
        
        # load salinity and temp from hycom
        temp, tlat, tlon, ttime, tdep = \
        temp_salinity_source.load_temp(south=ymin, north=ymax, west=xmin, east=xmax, 
                                       start=start, end=end, top=top, bottom=bottom)

        salinity, slat, slon, stime, sdep = \
        temp_salinity_source.load_temp(south=ymin, north=ymax, west=xmin, east=xmax, 
                                       start=start, end=end, top=top, bottom=bottom)
        
        # need more wave data
        """
        try:
            wave, wlat, wlon, wtime = \
            wavesource2.load_windwaveheight(south=ymin, north=ymax, 
                                                 west=xmin, east=xmax, 
                                                 start=start, end=end)
        except Exception as e :
            print(f"no wave data for {(ymin, xmin)}", end='\t')
            print(e)
            continue
        """
        
        # instead, just using 5 as a constant wave height for now
        wave = np.array([5 for x in temp])
        
        oceantest = Ocean(salinity=salinity, temp=temp, bathy=bathy, wave=wave, wave_var='comb. wind wave swell height')
        oceantest.set_origin(lat_ref=tlat, lon_ref=tlon)
        
        sndspd = SoundSpeed(oceantest)._sound_speed(tlat, tlon, tdep, temp, salinity)
        
        print(sndspd, ymin, xmin)
